In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
spark.version

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz

In [ ]:
!ls -lh fhvhv_tripdata_2021-06.csv.gz

In [ ]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [ ]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv.gz')

In [ ]:
df = df.repartition(12)

df.write.parquet('data/pq/fhvhv/2021/06/')

In [ ]:
df = spark.read.parquet('data/pq/fhvhv/2021/06/')

**Q3**: How many taxi trips were there on June 15?

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

In [ ]:
df.registerTempTable('fhvhv_2021_06')

In [ ]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_2021_06
WHERE
    to_date(pickup_datetime) = '2021-06-15';
""").show()

**Q4**: Longest trip for each day

In [ ]:
df.columns

In [ ]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

In [ ]:
spark.sql("""
SELECT
    *
FROM 
    fhvhv_2021_06
ORDER BY
    2 DESC
LIMIT 10;
""").show()

In [ ]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 60) AS duration
FROM 
    fhvhv_2021_06
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

**Not hw**: Most frequent `dispatching_base_num`

How many stages this spark job has?



In [ ]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1)
FROM 
    fhvhv_2021_06
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

In [ ]:
df \
    .groupBy('dispatching_base_num') \
        .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show()

**Not hw**: Most common locations pair

In [ ]:
df_zones = spark.read.parquet('zones')

In [ ]:
df_zones.columns

In [ ]:
df.columns

In [ ]:
df_zones.registerTempTable('zones')

In [ ]:
spark.sql("""
SELECT
    CONCAT(pul.Zone, ' / ', dol.Zone) AS pu_do_pair,
    COUNT(1)
FROM 
    fhvhv_2021_06 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

**Q6**: Most frequent `pickupzone`


In [ ]:
spark.sql("""
SELECT
    CONCAT(pul.Zone) AS pickupzone,
    COUNT(1)
FROM 
    fhvhv_2021_06 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()